In [43]:
import csv, sys, time, requests
import obonet

import numpy as np
import pandas as pd
import networkx as nx

from pathlib import Path

UNICHEM_BASE = "https://www.ebi.ac.uk/unichem/api/v1"
CHEMBL_ATC = "https://www.ebi.ac.uk/chembl/adata/atc_class.json"
WD_SPARQL = "https://query.wikidata.org/sparql"

HEADERS_JSON = {"Accept": "application/json"}
WD_HEADERS = {"Accept": "application/sparql-results+json", "User-Agent": "open-atc-mapper/1.0"}



# Load Hetionet nodes

In [3]:
hetionet_nodes = pd.read_csv("data/hetionet-v1.0-nodes.tsv", sep="\t")
drugbank_ids = [row["id"].split("::")[1] for _, row in hetionet_nodes.iterrows() if row["kind"] == "Compound"]
drugbank_ids[:3]

['DB00014', 'DB00035', 'DB00050']

# Link DrugBank ID with ATC codes

In [4]:
def get_unichem_sources():
    r = requests.get(f"{UNICHEM_BASE}/sources", headers=HEADERS_JSON, timeout=30)
    r.raise_for_status()
    srcs = r.json()["sources"]
    s_by_name = {s["nameLong"].lower(): s["sourceID"] for s in srcs}
    # find DrugBank & ChEMBL ids (names can vary slightly)
    db_src = next(v for k,v in s_by_name.items() if "drugbank" in k)
    chembl_src = next(v for k,v in s_by_name.items() if "chembl" in k and "target" not in k)
    return db_src, chembl_src

def drugbank_to_chembl_via_unichem(dbid, db_src, chembl_src):
    payload = {"type":"sourceID","sourceID":db_src,"compound":dbid}
    r = requests.post(f"{UNICHEM_BASE}/compounds", json=payload, headers=HEADERS_JSON, timeout=60)
    if r.status_code == 404:  # not found
        return None
    r.raise_for_status()
    comps = r.json().get("compounds", [])
    if not comps:
        return None
    # each compound lists all sources; find the chembl id
    for comp in comps:
        for s in comp.get("sources", []):
            if s.get("id") == chembl_src:
                return s.get("compoundId")
    return None

def chembl_atc_for_molecule(chembl_id):
    r = requests.get(CHEMBL_ATC, params={"molecule_chembl_id": chembl_id}, headers=HEADERS_JSON, timeout=60)
    r.raise_for_status()
    data = r.json().get("atc_class", [])
    # prefer level5 where present; fall back to generic atc_code fields
    out = []
    for row in data:
        code = row.get("level5") or row.get("atc_code") or row.get("who_code")
        if code:
            out.append(code)
    return sorted(set(out))

def wikidata_atc_for_drugbanks(batch_dbids):
    # batch SPARQL with VALUES
    vals = " ".join(f'"{x}"' for x in batch_dbids)
    q = f"""
    SELECT ?drugbank ?atc WHERE {{
      VALUES ?drugbank {{ {vals} }}
      ?drug wdt:P715 ?drugbank .
      ?drug wdt:P267 ?atc .
    }}
    """
    r = requests.get(WD_SPARQL, params={"query": q, "format":"json"}, headers=WD_HEADERS, timeout=60)
    r.raise_for_status()
    res = r.json()["results"]["bindings"]
    out = {}
    for b in res:
        db = b["drugbank"]["value"]
        atc = b["atc"]["value"]
        out.setdefault(db, set()).add(atc)
    return {k: sorted(v) for k,v in out.items()}


In [ ]:
out_csv = "data/drugbank_atc.csv"
sleep = 0.3

db_src, chembl_src = get_unichem_sources()

results = {}  # dbid -> set(atc)
# 1) UniChem → ChEMBL → ATC
for dbid in drugbank_ids:
    try:
        chembl = drugbank_to_chembl_via_unichem(dbid, db_src, chembl_src)
        if chembl:
            atcs = chembl_atc_for_molecule(chembl)
            if atcs:
                results[dbid] = set(atcs)
        time.sleep(sleep)  # be gentle
    except Exception as e:
        # continue; will try Wikidata later
        pass

# 2) Wikidata fallback for those still missing
missing = [d for d in drugbank_ids if d not in results]
B = 200  # WDQS likes modest batches
for i in range(0, len(missing), B):
    batch = missing[i:i+B]
    try:
        wd_map = wikidata_atc_for_drugbanks(batch)
        for dbid, atcs in wd_map.items():
            results.setdefault(dbid, set()).update(atcs)
        time.sleep(1.0)
    except Exception:
        pass

# write rows: one (dbid, atc) per line
with open(out_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["drugbank_id","atc_code"])
    for dbid in drugbank_ids:
        if dbid in results and results[dbid]:
            for code in sorted(results[dbid]):
                w.writerow([dbid, code])
        else:
            w.writerow([dbid, ""])  # no ATC found

# Create ATC drug hierarchy

In [6]:
out_csv = "data/drugbank_atc.csv"
drug_atc_df = pd.read_csv(out_csv, dtype=str).dropna()
drug_atc_df.describe()

,drugbank_id,atc_code
count,1742,1742
unique,1324,1741
top,DB01234,J01XB01
freq,11,2


In [7]:
ATC_ROOT = "drug"

def atc_parent(code: str) -> str:
    code = code.strip()
    L = len(code)
    if L <= 1:
        return ATC_ROOT
    if L == 3:
        return code[:1]
    if L == 4:
        return code[:3]
    if L == 5:
        return code[:4]
    if L >= 7:
        return code[:5]
    return code[:-1] if L > 1 else ATC_ROOT

def build_atc_tree(atc_codes: set[str]):
    tree = set()
    stack = atc_codes.copy()
    while stack:
        c = stack.pop()
        p = atc_parent(c)
        if p and p != c:
            tree.add((c, p))
            if p not in atc_codes and p != ATC_ROOT:
                atc_codes.add(p)
                stack.add(p)
    # connect single-letter tops to ATC:ROOT
    for c in list(atc_codes):
        if len(c) == 1:
            tree.add((c, ATC_ROOT))
    return tree

In [8]:
atc_tree = build_atc_tree(set(drug_atc_df["atc_code"]))
atc_hierarchy = pd.DataFrame(atc_tree, columns=["child", "parent"])

hetionet_atc = drug_atc_df.rename(columns={"drugbank_id": "child", "atc_code": "parent"})
hetionet_atc["child"] = "Compound::" + hetionet_atc.child

drug_hierarchy = pd.concat([atc_hierarchy, hetionet_atc])
drug_path = Path('data/hetionet_atc_drug_hierarchy.tsv')
drug_hierarchy.to_csv(drug_path, sep="\t", index=False)

print(drug_hierarchy[:-3])


                  child   parent
0               D07XB05    D07XB
1               B02BA02    B02BA
2               M02AA07    M02AA
3               D07AC01    D07AC
4               J01DC09    J01DC
...                 ...      ...
1961  Compound::DB09009  N01BB08
1962  Compound::DB09014  N05BB02
1964  Compound::DB09017  N05CD09
1965  Compound::DB09018  A03FA04
1966  Compound::DB09019  R05CB02

[4226 rows x 2 columns]


# Create drug hierarchy

Download DOID OBO from https://obofoundry.org/ontology/doid.html

In [12]:
def build_doid_edges(
    doid_obo_path: str, 
    keep_only: set[str] = None,
    root_id: str = "DOID:4",
    root_label: str = "disease"
    ):
    g = obonet.read_obo(doid_obo_path)
    G = nx.DiGraph()
    for u, v, d in g.edges(data=True):
        if d.get('relation', 'is_a') == 'is_a':
            G.add_edge(u, v)  # child -> parent
    
    # Filter and keep only the specified diseases
    if keep_only:
        # keep nodes that are in keep_only or ancestors of those
        keep = set()
        for n in keep_only:
            if n in G:
                keep.add(n)
                keep |= nx.ancestors(G, n)
        G = G.subgraph(keep).copy()

    # Create edges
    edges = {(u, v) for u, v in G.edges()}

    # Add root node
    edges.add((root_id, root_label))

    return edges

In [13]:
disease_hierarchy_path = "data/doid.obo"

hetionet_doid = hetionet_nodes.query('kind == "Disease"')["id"].to_list()
hetionet_doid = set([d.split("::")[1] for d in hetionet_doid])

doid_edges = build_doid_edges(disease_hierarchy_path, keep_only=hetionet_doid)
doid_edges_df = pd.DataFrame(doid_edges, columns=["child", "parent"])
doid_edges_df.to_csv("data/hetionet_doid_hierarchy.tsv", index=False)

# Combine drug and disease heirarchy

In [41]:
hetionet_hierarchy = pd.concat([drug_hierarchy, doid_edges_df])
hetionet_hierarchy.to_csv("data/hetionet_hierarchy.csv", index=False)


# Prepare Hetionet network file for DREAMwalk

## Prepare edges

In [33]:
hetionet_edges = pd.read_csv("data/hetionet-v1.0-edges.sif.gz", sep="\t")

In [38]:
# select only the edge types included in the original study
selected_metaedge = ['CbG', 'CdG', 'CtD', 'CuG', 'DaG', 'DdG', 'DuG', 'GcG', 'GiG', 'GpPW', 'Gr>G']
filtered_hetionet_edges = hetionet_edges.query('metaedge in @selected_metaedge')

In [39]:
unique_edges = filtered_hetionet_edges.metaedge.unique()
edge_type_id_map = {edge: idx+1 for idx, edge in enumerate(unique_edges)}

In [56]:
dreamwalk_hetionet_network = pd.DataFrame({
    "source": filtered_hetionet_edges.source,
    "target": filtered_hetionet_edges.target,
    "edge_type": filtered_hetionet_edges.metaedge.map(edge_type_id_map),
    "weight": 1.,
    "edge_id": range(1, len(filtered_hetionet_edges)+1)
})
dreamwalk_hetionet_network.to_csv("data/dreamwalk_hetionet_network.txt", header=False, index=False, sep="\t")


## Prepare nodes

In [53]:
# Select onlly certain node types included in the original DREAMwalk study
print(f"Total number of nodes: {len(hetionet_nodes)}")
filtered_hetionet_nodes = hetionet_nodes.query('kind == "Compound" | kind == "Disease" | kind == "Gene" | kind == "Pathway"')
print(f"Number of nodes after filtering node types: {len(filtered_hetionet_nodes)}")
filtered_hetionet_nodes = filtered_hetionet_nodes.query('id in @dreamwalk_hetionet_network.source | id in @dreamwalk_hetionet_network.target')
print(f"Number of nodes after filtering existing edges: {len(filtered_hetionet_nodes)}")

Total number of nodes: 47031
Number of nodes after filtering node types: 24456
Number of nodes after filtering existing edges: 21899


In [54]:
dreamwalk_hetionet_nodetypes = pd.DataFrame({
    "node": filtered_hetionet_nodes.id,
    "type": filtered_hetionet_nodes.kind,
})
dreamwalk_hetionet_nodetypes.to_csv("data/dreamwalk_hetionet_nodetypes.txt", header=False, index=False, sep="\t")

# Get positive and negative edges

In [47]:
def sample_negatives(all_drugs, all_diseases, pos_set, target_n_neg):
	rng = np.random.default_rng(42)
	negs = []
	CHUNK = min(2_000, target_n_neg * 10)
	while len(negs) < target_n_neg:
		c = rng.choice(all_drugs, size=CHUNK)
		d = rng.choice(all_diseases, size=CHUNK)
		cand = list(map(tuple, np.c_[c, d]))
		cand = [c for c in cand if c not in pos_set]
		negs.extend(cand)
	negs = negs[:target_n_neg]
	neg_df = pd.DataFrame(negs, columns=["Compound", "Disease"])
	neg_df["label"] = 0
	return neg_df

In [57]:
pos = [(row.source, row.target) for _, row in dreamwalk_hetionet_network.iterrows() if row.edge_type == edge_type_id_map['CtD']]
pos_set = set(pos)
pos = pd.DataFrame(pos, columns=["Compound", "Disease"])
pos["label"] = 1

all_drugs = filtered_hetionet_nodes.query('kind == "Compound"').id.unique()
all_diseases = filtered_hetionet_nodes.query('kind == "Disease"').id.unique()

neg = sample_negatives(all_drugs, all_diseases, pos_set, len(pos_set) * 5)

drug_disease_associations = pd.concat([pos, neg], ignore_index=True)

In [58]:
drug_disease_associations.to_csv("data/dreamwalk_hetionet_drug_disease_associations.tsv", index=False, sep="\t")